In [1]:
using Distributed

using Pkg


@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100




100

In [3]:
 runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
 atomtype="Cu"
# pot = AbstractMLPotential(runnerdir,atomtype)
# RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)

"Cu"

In [4]:
pot_vector = []

for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_vector,potential)
end

mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1] ; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
    

results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

 -------------------------------------------------------------
 ---------------------- Welcome to the -----------------------
     RuNNer Neural Network Energy Representation - RuNNer     
 ----------  (c) 2008-2022 Prof. Dr. Joerg Behler   ----------
 ----------  Georg-August-Universitaet Goettingen   ----------
 ----------           Theoretische Chemie           ----------
 ----------              Tammannstr. 6              ----------
 ----------        37077 Goettingen, Germany        ----------
 -------------------------------------------------------------
 -------------------------------------------------------------
  This program is free software: you can redistribute it and/or modify it
  under the terms of the GNU General Public License as published by the 
  Free Software Foundation, either version 3 of the License, or 
  (at your option) any later version.
    
  This program is distributed in the hope that it will be useful, but 
  WITHOUT ANY WARRANTY; without even the imp

   14 Cu  3  Cu Cu   0.000100  -1.000000   2.000000  11.338000
   15 Cu  3  Cu Zn   0.000100  -1.000000   2.000000  11.338000
   16 Cu  3  Zn Zn   0.000100  -1.000000   2.000000  11.338000
   17 Cu  3  Cu Cu   0.003000  -1.000000   1.000000  11.338000
   18 Cu  3  Cu Zn   0.003000  -1.000000   1.000000  11.338000
   19 Cu  3  Zn Zn   0.003000  -1.000000   1.000000  11.338000
   20 Cu  3  Cu Cu   0.003000  -1.000000   2.000000  11.338000
   21 Cu  3  Cu Zn   0.003000  -1.000000   2.000000  11.338000
   22 Cu  3  Zn Zn   0.003000  -1.000000   2.000000  11.338000
   23 Cu  3  Cu Cu   0.008000  -1.000000   1.000000  11.338000
   24 Cu  3  Cu Zn   0.008000  -1.000000   1.000000  11.338000
   25 Cu  3  Zn Zn   0.008000  -1.000000   1.000000  11.338000
   26 Cu  3  Cu Cu   0.008000  -1.000000   2.000000  11.338000
   27 Cu  3  Cu Zn   0.008000  -1.000000   2.000000  11.338000
   28 Cu  3  Zn Zn   0.008000  -1.000000   2.000000  11.338000
   29 Cu  3  Cu Cu   0.008000   1.000000   2.000000  11

 -------------------------------------------------------------
 ---------------------- Welcome to the -----------------------
     RuNNer Neural Network Energy Representation - RuNNer     
 ----------  (c) 2008-2022 Prof. Dr. Joerg Behler   ----------
 ----------  Georg-August-Universitaet Goettingen   ----------
 ----------           Theoretische Chemie           ----------
 ----------              Tammannstr. 6              ----------
 ----------        37077 Goettingen, Germany        ----------
 -------------------------------------------------------------
 -------------------------------------------------------------
  This program is free software: you can redistribute it and/or modify it
  under the terms of the GNU General Public License as published by the 
  Free Software Foundation, either version 3 of the License, or 
  (at your option) any later version.
    
  This program is distributed in the hope that it will be useful, but 
  WITHOUT ANY WARRANTY; without even the imp

   14 Cu  3  Cu Cu   0.000100  -1.000000   2.000000  11.338000
   15 Cu  3  Cu Zn   0.000100  -1.000000   2.000000  11.338000
   16 Cu  3  Zn Zn   0.000100  -1.000000   2.000000  11.338000
   17 Cu  3  Cu Cu   0.003000  -1.000000   1.000000  11.338000
   18 Cu  3  Cu Zn   0.003000  -1.000000   1.000000  11.338000
   19 Cu  3  Zn Zn   0.003000  -1.000000   1.000000  11.338000
   20 Cu  3  Cu Cu   0.003000  -1.000000   2.000000  11.338000
   21 Cu  3  Cu Zn   0.003000  -1.000000   2.000000  11.338000
   22 Cu  3  Zn Zn   0.003000  -1.000000   2.000000  11.338000
   23 Cu  3  Cu Cu   0.008000  -1.000000   1.000000  11.338000
   24 Cu  3  Cu Zn   0.008000  -1.000000   1.000000  11.338000
   25 Cu  3  Zn Zn   0.008000  -1.000000   1.000000  11.338000
   26 Cu  3  Cu Cu   0.008000  -1.000000   2.000000  11.338000
   27 Cu  3  Cu Zn   0.008000  -1.000000   2.000000  11.338000
   28 Cu  3  Zn Zn   0.008000  -1.000000   2.000000  11.338000
   29 Cu  3  Cu Cu   0.008000   1.000000   2.000000  11

   86 Zn  3  Cu Cu   0.080000   1.000000   4.000000  11.338000
   87 Zn  3  Cu Zn   0.080000   1.000000   4.000000  11.338000
   88 Zn  3  Zn Zn   0.080000   1.000000   4.000000  11.338000
 -------------------------------------------------------------
 Prediction Mode for all configurations
 -------------------------------------------------------------
 NNconfiguration:         1    Number of atoms:        13
 -------------------------------------------------------------
 begin
 atom      -0.00000005     -0.00000005     -0.00000003 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -0.00000752     -0.00000001      6.73662421 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom       1.86195381     -5.73051487      3.01271201 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -4.87467046     -3.54165303      3.01270453 Cu       0.00000000      0.00000000      0.00

...

  20     0.00015091    4.87827144    1.74417092    4.87812052
  21     0.00000000    2.55433510    0.41201677    2.55433510
  22     0.00000000    2.63292295    0.07998148    2.63292295
  23     0.00036475    6.84954457    2.55727811    6.84917982
  24     0.00000000    3.37663635    0.60046955    3.37663635
  25     0.00000000    3.60734063    0.11012497    3.60734063
  26     0.00004682    2.87878694    1.00202413    2.87874011
  27     0.00000000    1.57328033    0.23637910    1.57328033
  28     0.00000000    1.54989710    0.04567094    1.54989710
  29     0.00330383   14.20003263    5.51399542   14.19672880
  30     0.00000000    6.07940896    1.27630877    6.07940896
  31     0.00000000    6.17826499    0.20316295    6.17826499
  32     0.00081609    9.46250663    3.58082950    9.46169054
  33     0.00000000    4.38136788    0.83294204    4.38136788
  34     0.00000000    4.39433923    0.13245838    4.39433923
  35     0.00000924    1.43539576    0.47859750    1.43538652
  36    

 -------------------------------------------------------------
 average eshort per atom, eshortmax        -0.0396064007       -0.0510778715
 -------------------------------------------------------------
 -------------------------------------------------------------
 NN sum of free atom energies, short range and electrostatic energy for configuration         1
 NN sum of free atom energies         0.00000000 Ha
 NN short range energy               -0.51488321 Ha
 NN electrostatic energy              0.00000000 Ha
 NN vdw dispersion energy             0.00000000 Ha
 NNenergy                            -0.51488321 Ha
 -------------------------------------------------------------
 NN atomenergies for configuration            1
NNatomenergy      1 Cu      -0.065355
NNatomenergy      2 Cu      -0.037461
NNatomenergy      3 Cu      -0.037461
NNatomenergy      4 Cu      -0.037461
NNatomenergy      5 Cu      -0.037461
NNatomenergy      6 Cu      -0.037461
NNatomenergy      7 Cu      -0.037461
N

Excessive output truncated after 524536 bytes.

   6   G  t  t   
   7   G  t  t   
   8   G  t  t   
   9   G  t  t   
  10   G  t  t   
  11   G  t  t   
  12   G  t  t   
  13   G  t  t   
  14   G  t  t   
  15   G  t  t   
  16   G  t  t   
  17   G  t  t   
  18   G  t  t   
  19   G  t  t   
  20   G  t  t   
  21   G
  22   G
  23   G
  24   G
  25   G
  26   G
  27   G
  28   G
  29   G
  30   G
  31   G
  32   G
  33   G
  34   G
  35   G
  36   G
  37   G
  38   G
  39   G
  40   G
  41   G
  42   G
  43   G
  44   G
  45   G
  46   G
  47   G
  48   G
  49   G
  50   G
  51   G
  52   G
  53   G
  54   G
  55   G
  56   G
  57   G
  58   G
  59   G
  60   G
  61   G
  62   G
  63   G
  64   G
  65   G
  66   G
  67   G
  68   G
  69   G
  70   G
  71   G
  72   G
  73   G
  74   G
  75   G
  76   G
  77   G
  78   G
  79   G
  80   G
  81   G
  82   G
  83   G
  84   G
  85   G
  86   G
  87   G
  88   G
 -------------------------------------------------------------
 ------------------------------------------------------

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [5]:
parallel_states,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-0.09850579885821831, -0.005157842443408889, 0.1248460009153229], [0.13383528664474098, 0.003456137642791279, 7.04811993740721], [2.0184164854865636, -5.796311048825306, 3.2924251793792787], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.907591293403073, 3.7007720913548483, 3.0607727753581417], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.316424044033252, -0.16341367787579042, 3.1426256436771247], [-1.868257199897207, -6.026474706790241, -2.9984610601378052], [-6.323036830175243, 0.11495671003974081, -3.103324778832583], [-1.8645412856453434, 5.875118034708346, -3.062523047135528], [5.0988286730956505, 3.6457034703958944, -3.2180436114954114], [4.977810402656452, -3.6425922295152917, -3.0897871786350253], [-0.146681160069665, 0.011217527401057414, -6.9726792

In [6]:
function testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner (generic function with 1 method)

In [7]:
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]
#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu55 = copperconstant*ico_55
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886

# length(pos_cu55) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_cu55 = SphericalBC(radius=14*AtoBohr)   #5.32 Angstrom
bc_cu13 = SphericalBC(radius=8*AtoBohr)
#starting configuration
start_config = Config(pos_cu55, bc_cu55)
#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1]; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
mc_cycles = 50 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

MCParams(50, 50, 1, 16, 13, 100)

In [14]:
parallel_states,pot_vec,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 55, SphericalBC{Float64}}[MCState{Float64, 55, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{55, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[0.22417984421425763, -0.23295678281238164, 0.3832409985212912], [0.17494579141215671, -0.15857243233776364, 4.571071587784498], [1.2682857540970016, -3.8718173571575685, 1.8184835193386075], [-3.168341664786904, -2.3136927990199543, 1.9900695432053928], [-3.1203148112178067, 2.512998240669963, 1.8833607315027596], [1.4751185144438872, 3.5875658993086015, 1.6544924363612878], [4.348397233359417, 0.08822768972505132, 1.9315037361915914], [-1.1722088299229303, -3.9350370153118863, -1.8319928126131932], [-4.275386839659444, -0.06957474531729634, -2.133693812559925], [-1.4472754101382446, 3.7662895740465228, -1.8378976153000615]  …  [2.0744963372369405, 6.384583971948036, -4.148977939714466], [6.5594487104586, 4.765688045720408, -4.053952548518178], [6.713184197946128, -2.1916113212552402e-5, -4.14

In [9]:
function testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner2 (generic function with 1 method)

In [15]:
@benchmark testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 88.506 s (0.01% GC) to evaluate,
 with a memory estimate of 151.65 MiB, over 1892130 allocations.

In [16]:
@benchmark testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 23.860 s (0.10% GC) to evaluate,
 with a memory estimate of 171.28 MiB, over 2669923 allocations.